# Random Forest Models: Classification and Regression

This notebook describes how to implement Zero-Inflated Random Forest models to predict board counts at each bus stop.

 * Requiered libraries:

In [7]:
library(randomForest)
library(mlbench)
library(caret)
library(e1071)
library(dplyr)
library(tidyr)
library(readr)
library(ranger)
library(rFerns)
library(ordinalForest)
library(RRF)


Attaching package: ‘ranger’


The following object is masked from ‘package:randomForest’:

    importance


Registered S3 method overwritten by 'RRF':
  method      from        
  plot.margin randomForest

RRF 1.9.1

Type rrfNews() to see new features/changes/bug fixes.


Attaching package: ‘RRF’


The following object is masked from ‘package:ranger’:

    importance


The following object is masked from ‘package:dplyr’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin


The following objects are masked from ‘package:randomForest’:

    classCenter, combine, getTree, grow, importance, margin, MDSplot,
    na.roughfix, outlier, partialPlot, treesize, varImpPlot, varUsed




## Zero-Inflated Random Forest Model

The proposed Zero-Inflated Random Forest model can be described as follows:

    * Classification: It learns a classifier to predict counts (> 0) and zero counts using Random Ferns.
    * Regression: It takes the *predicted counts* data to learn a regression model for these counts using Random Forest.
    
The following runs the Zero-Inflated Random Forest model. It requires the following inputs:

    * `route_id == rt`.
    * `direction_id == di`.
    * `stop_id == st`.
    * `part == part` (Pre or port-lockdown dataset).

In [143]:
RF_Ferns_and_Ranger <- function(rt, di, st, part){
    path = paste0('data', '/', 'jmartinez', '/', 'Data_for_RF_Models', '/', 'Board_Counts', '/',
                  paste('route', rt, sep = '_'), '/', paste('direction', di, sep = ''), '/',
                  paste('bus_stop', st, sep = '_'), '/')
    if(part == 'pre'){
        file_path_train = paste(path, 'pre_lock_train_data.csv', sep = '/')
        file_path_test = paste(path, 'pre_lock_test_data.csv', sep = '/')
        
        board_train = read_csv(file_path_train)
        board_test = read_csv(file_path_test)
        
        board_train$month = factor(board_train$month)
        board_train$service_kind = factor(board_train$service_kind)
        board_train$hour = factor(board_train$hour)

        board_test$month = factor(board_test$month)
        board_test$service_kind = factor(board_test$service_kind)
        board_test$hour = factor(board_test$hour)
    }
    else if(part == 'post'){
        
        file_path_train = paste(path, 'post_lock_train_data.csv', sep = '/')
        file_path_test = paste(path, 'post_lock_test_data.csv', sep = '/')
        
        board_train = read_csv(file_path_train)
        board_test = read_csv(file_path_test)
        
        board_train$month = factor(board_train$month)
        board_train$service_kind = factor(board_train$service_kind)
        board_train$hour = factor(board_train$hour)

        board_test$month = factor(board_test$month)
        board_test$service_kind = factor(board_test$service_kind)
        board_test$hour = factor(board_test$hour)
    }
    else{
        file_path_train = paste(path, 'train_data.csv', sep = '/')
        file_path_test = paste(path, 'test_data.csv', sep = '/')
        
        board_train = read_csv(file_path_train)
        board_test = read_csv(file_path_test)
        
        board_train$month = factor(board_train$month)
        board_train$service_kind = factor(board_train$service_kind)
        board_train$hour = factor(board_train$hour)

        board_test$month = factor(board_test$month)
        board_test$service_kind = factor(board_test$service_kind)
        board_test$hour = factor(board_test$hour)
    }
    
    train_month_levels = length(levels(board_train$month))
    train_service_kind_levels = length(levels(board_train$service_kind))
    train_hour_levels = length(levels(board_train$hour))    
    
    board_test = board_test %>%
        filter(hour %in% intersect(unique(board_test$hour), unique(board_train$hour)))
    
    if(train_month_levels > 1){
        if(train_service_kind_levels > 1){
            if(train_hour_levels > 1){
                board_train = board_train 
                
                board_test = board_test 
            }
            else{
                board_train = board_train %>%
                    summarise(board_count, service_kind, month, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
                
                board_test = board_test %>%
                    summarise(board_count, service_kind, month, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
            }
        }
        else{
            if(train_hour_levels > 1){
                
                board_train = board_train %>%
                    summarise(board_count, hour, month, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
                
                board_test = board_test %>%
                    summarise(board_count, hour, month, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
            }
        }
    }
    else{
        if(train_service_kind_levels > 1){
            if(train_hour_levels > 1){
                board_train = board_train %>%
                    summarise(board_count, hour, service_kind, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
                
                board_test = board_test %>%
                    summarise(board_count, hour, service_kind, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
            }
            else{
                board_train = board_train %>%
                    summarise(board_count, service_kind, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
                
                board_test = board_test %>%
                    summarise(board_count, service_kind, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
            }
        }
        else{
            if(train_hour_levels > 1){
                board_train = board_train %>%
                    summarise(board_count, hour, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
                
                board_test = board_test %>%
                    summarise(board_count, hour, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
            }
            else{
                board_train = board_train %>%
                    summarise(board_count, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
                
                board_test = board_test %>%
                    summarise(board_count, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
            }
        }
    }
    
    n_row_train = nrow(board_train)
    
    if(n_row_train < 60){
        
        return('Insufficient data for analysis!')
    }
    else{
        y_clf_train = board_train$board_count
        y_clf_train = factor(if_else(y_clf_train == 0, 0, 1))
        
        y_clf_test = board_test$board_count
        y_clf_test = factor(if_else(y_clf_test == 0, 0, 1))
        
        Board_train_clf <- data.frame(cbind(y_clf_train, board_train[, -c(1)]))
        Board_test_clf <- data.frame(cbind(y_clf_test, board_test[, -c(1)]))
        
        #---------------------------------------------------------------------------------
        # Training characteristics for model tuning:
        #---------------------------------------------------------------------------------
        
        control <- trainControl(method='repeatedcv', 
                                number=10, 
                                repeats=3,
                                search = 'random')
        
        #---------------------------------------------------------------------------------
        # Classification using Random Ferns:
        #---------------------------------------------------------------------------------
        
        set.seed(1)
        rf_random <- train(y_clf_train ~ .,
                           data = Board_train_clf,
                           method = 'rFerns',
                           metric = 'Accuracy',
                           tuneLength  = 20, 
                           trControl = control)
        
        RF_Ferns <- print(rf_random)
                
        rf_random_pred <- predict(rf_random, newdata = Board_test_clf)
        
        rf_random_conf_mat <- confusionMatrix(y_clf_test, rf_random_pred)
        rf_random_conf_mat <- data.frame(rf_random_conf_mat[4])
        colnames(rf_random_conf_mat) <- c('Value')
        # Index for regression data:
        
        index_for_reg <- which(rf_random_pred == '1', arr.ind = T)
        #---------------------------------------------------------------------------------
        # Regression Model using Ranger:
        
                
        set.seed(1)
        rf_reg_ranger <- train(board_count ~ .,
                               data = board_train,
                               method = 'ranger',
                               metric = 'RMSE',
                               tuneLength  = 20, 
                               trControl = control)
        
        RF_Ranger <- print(rf_reg_ranger)
                
        #----------------------------------------------------------------------------------
        # Validation:
        Board_Test_Val = board_test
        nrow_test = (1:nrow(board_test))
        
        Board_Test_Val$index = nrow_test
        Board_test_reg = Board_Test_Val[index_for_reg, ]
        
        rf_reg_ranger_pred <- predict(rf_reg_ranger, newdata = Board_test_reg)
        Board_test_reg$Ranger_Pred = rf_reg_ranger_pred
        
        Board_Test_Val = left_join(Board_Test_Val, Board_test_reg, by = 'index')
        
        Board_Test_Val = Board_Test_Val %>%
            mutate(RF_Pred = if_else(is.na(Ranger_Pred) == T, 0, Ranger_Pred))
        
        board_test$RF_Pred = Board_Test_Val$RF_Pred
                
        RF_test_RMSE = sqrt(mean((board_test$board_count - board_test$RF_Pred)^{2}))
    }
    
    if(part == 'pre'){
        file_path_clf = paste(path, 'pre_lock_RF_Fern.txt', sep = '/')
        file_path_clf_conf_mat = paste(path, 'pre_Conf_Mat_RF_Fern.csv', sep = '/')
        
        file_path_reg = paste(path, 'pre_lock_RF_Reg.txt', sep = '/')
        file_path_RF_Chart = paste(path, 'pre_RF_Chart.csv', sep = '/')
        
        final_clf_model = paste(path, 'Pre_Random_Ferns_model.rds')
        final_reg_model = paste(path, 'Pre_Random_Forest_RANGER_model.rds')
        
        write.table(RF_Ferns, file_path_clf)
        write.csv(rf_random_conf_mat, file_path_clf_conf_mat)
        
        write.table(RF_Ranger, file_path_reg)
        write.csv(board_test, file_path_RF_Chart)
        
        saveRDS(rf_random, final_clf_model)
        saveRDS(rf_reg_ranger, final_reg_model)
        
    }
    else if(part == 'post'){
        
        file_path_clf = paste(path, 'post_lock_RF_Fern.txt', sep = '/')
        file_path_clf_conf_mat = paste(path, 'post_Conf_Mat_RF_Fern.csv', sep = '/')
        
        file_path_reg = paste(path, 'post_lock_RF_Reg.txt', sep = '/')
        file_path_RF_Chart = paste(path, 'post_RF_Chart.csv', sep = '/')
        
        final_clf_model = paste(path, 'Post_Random_Ferns_model.rds')
        final_reg_model = paste(path, 'Post_Random_Forest_RANGER_model.rds')
        
        write.table(RF_Ferns, file_path_clf)
        write.csv(rf_random_conf_mat, file_path_clf_conf_mat)
        
        write.table(RF_Ranger, file_path_reg)
        write.csv(board_test, file_path_RF_Chart)
        
        saveRDS(rf_random, final_clf_model)
        saveRDS(rf_reg_ranger, final_reg_model)
    }
    else{
        file_path_clf = paste(path, 'RF_Fern.txt', sep = '/')
        file_path_clf_conf_mat = paste(path, 'Conf_Mat_RF_Fern.csv', sep = '/')
        
        file_path_reg = paste(path, 'RF_Reg.txt', sep = '/')
        file_path_RF_Chart = paste(path, 'pre_RF_Chart.csv', sep = '/')
        
        final_clf_model = paste(path, 'Random_Ferns_model.rds')
        final_reg_model = paste(path, 'Random_Forest_RANGER_model.rds')
        
        write.table(RF_Ferns, file_path_clf)
        write.csv(rf_random_conf_mat, file_path_clf_conf_mat)
        
        write.table(RF_Ranger, file_path_reg)
        write.csv(board_test, file_path_RF_Chart)
        
        saveRDS(rf_random, final_clf_model)
        saveRDS(rf_reg_ranger, final_reg_model)
    }
    
    
    return('Done!')
}

In [149]:
RF_Ferns_and_Ranger('4', '0', '1353', 'post')


── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




Random Ferns 

477 samples
  7 predictor
  2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 429, 429, 429, 429, 429, 429, ... 
Resampling results across tuning parameters:

  depth  Accuracy   Kappa    
   1     0.9274189  0.8261198
   2     0.9459799  0.8687012
   3     0.9385282  0.8479907
   5     0.9223781  0.8114063
   6     0.9287038  0.8239197
   7     0.9239006  0.8157220
   9     0.9168823  0.7988998
  10     0.9211370  0.8077262
  11     0.9168527  0.7989080
  13     0.9119898  0.7880959
  14     0.9204136  0.8071338
  15     0.8987203  0.7601983

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was depth = 2.
Random Forest 

477 samples
  7 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 431, 430, 429, 427, 429, 430, ... 
Resampling results across tuning parameters:

  min.node.size  m

[1] "Done!"

# For-Loop by route and direction

In [106]:
Pre_lock = read_csv('data/jmartinez/Transit_Data/Pre_lock.csv')


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  scheduled_arrival_time = col_datetime(format = ""),
  actual_arrival_time = col_datetime(format = ""),
  direction_desc = col_character(),
  service_period = col_character(),
  date = col_date(format = ""),
  scheduled_datetime = col_datetime(format = ""),
  actual_arrival_datetime = col_datetime(format = ""),
  trip_start_time = col_datetime(format = ""),
  trip_date = col_date(format = ""),
  service_kind = col_character()
)
ℹ Use `spec()` for the full column specifications.




In [128]:
Post_lock = read_csv('data/jmartinez/Transit_Data/Post_lock.csv')


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  scheduled_arrival_time = col_datetime(format = ""),
  actual_arrival_time = col_datetime(format = ""),
  direction_desc = col_character(),
  service_period = col_character(),
  date = col_date(format = ""),
  scheduled_datetime = col_datetime(format = ""),
  actual_arrival_datetime = col_datetime(format = ""),
  trip_start_time = col_datetime(format = ""),
  trip_date = col_date(format = ""),
  service_kind = col_character()
)
ℹ Use `spec()` for the full column specifications.




In [100]:
head(Pre_lock)

trip_id,scheduled_arrival_time,actual_arrival_time,stop_id,stop_sequence,stop_lat,stop_lon,route_id,direction_id,board_count,⋯,actual_arrival_datetime,trip_start_time,day_of_week,trip_date,hour,year,month,Estimated_Temp,Estimated_Precip,service_kind
<dbl>,<dttm>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dttm>,<dttm>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
132994,1900-01-01 04:55:00,1900-01-01 04:52:00,1351,1,35.05266,-85.30972,1,0,1,⋯,2019-01-02 04:52:00,1900-01-01 04:55:00,2,2019-01-02,4,2019,1,8.350000,0,weekday
132994,1900-01-01 04:55:41,1900-01-01 04:56:00,12,2,35.05070,-85.30953,1,0,0,⋯,2019-01-02 04:56:00,1900-01-01 04:55:00,2,2019-01-02,4,2019,1,8.343167,0,weekday
132994,1900-01-01 04:56:20,1900-01-01 04:57:00,1555,3,35.04883,-85.30955,1,0,0,⋯,2019-01-02 04:57:00,1900-01-01 04:55:00,2,2019-01-02,4,2019,1,8.336667,0,weekday
132994,1900-01-01 04:56:35,1900-01-01 04:57:00,1579,4,35.04812,-85.30955,1,0,0,⋯,2019-01-02 04:57:00,1900-01-01 04:55:00,2,2019-01-02,4,2019,1,8.334167,0,weekday
132994,1900-01-01 04:57:22,1900-01-01 04:57:00,1354,5,35.04598,-85.30952,1,0,0,⋯,2019-01-02 04:57:00,1900-01-01 04:55:00,2,2019-01-02,4,2019,1,8.326333,0,weekday
132994,1900-01-01 04:57:35,1900-01-01 04:57:00,1353,6,35.04538,-85.30945,1,0,0,⋯,2019-01-02 04:57:00,1900-01-01 04:55:00,2,2019-01-02,4,2019,1,8.324167,0,weekday


In [107]:
dim(Pre_lock)

[1] 8602818      26

In [110]:
Pre_lock$trip_id = as.character(Pre_lock$trip_id)
Pre_lock$stop_id = as.character(Pre_lock$stop_id)
Pre_lock$route_id = as.character(Pre_lock$route_id)
Pre_lock$direction_id = as.character(Pre_lock$direction_id)
Pre_lock$hour = factor(Pre_lock$hour)
Pre_lock$month = factor(Pre_lock$month)

In [129]:
Post_lock$trip_id = as.character(Post_lock$trip_id)
Post_lock$stop_id = as.character(Post_lock$stop_id)
Post_lock$route_id = as.character(Post_lock$route_id)
Post_lock$direction_id = as.character(Post_lock$direction_id)
Post_lock$hour = factor(Post_lock$hour)
Post_lock$month = factor(Post_lock$month)

In [113]:
bus_stops_r4_dir1_pre = Pre_lock %>%  
    filter(route_id == '4', direction_id == '1') %>%
    pull(stop_id)

In [130]:
bus_stops_r4_dir1_post = Post_lock %>%  
    filter(route_id == '4', direction_id == '1') %>%
    pull(stop_id)

In [131]:
bus_stops_r4_dir1_pre = unique(bus_stops_r4_dir1_pre)
bus_stops_r4_dir1_post = unique(bus_stops_r4_dir1_post)

In [137]:
for(i in bus_stops_r4_dir1_post[137:length(bus_stops_r4_dir1_post)]){
    print(i)
    RF_Ferns_and_Ranger('4', '1', i, 'post')
    print('Done!')
    print('_/|_/|_')
}

[1] "2148"



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




Random Ferns 

108 samples
  7 predictor
  2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 98, 97, 98, 96, 97, 97, ... 
Resampling results across tuning parameters:

  depth  Accuracy   Kappa    
   1     0.9143434  0.8209537
   2     0.9146465  0.8233835
   3     0.9213131  0.8368290
   5     0.9083333  0.8109977
   6     0.9055556  0.8027743
   7     0.9028283  0.7973770
   9     0.8776768  0.7519671
  10     0.8870707  0.7668673
  11     0.8716162  0.7398755
  13     0.8591919  0.7192164
  14     0.8531313  0.7057861
  15     0.8473232  0.6987384

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was depth = 3.
Random Forest 

108 samples
  7 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 97, 98, 97, 96, 98, 97, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  splitru


── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




[1] "Done!"
[1] "_/|_/|_"
[1] "2245"


...

[1] "Done!"
[1] "_/|_/|_"
[1] "2198"



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




[1] "Done!"
[1] "_/|_/|_"
[1] "808"



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_logical(),
  mean_precip = col_logical(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_logical(),
  mean_precip = col_logical(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




[1] "Done!"
[1] "_/|_/|_"


## Route 1

### Pre-lockdown

In [139]:
bus_stops_r1_dir1_pre = Pre_lock %>%  
    filter(route_id == '1', direction_id == '1') %>%
    pull(stop_id)

bus_stops_r1_dir1_pre = unique(bus_stops_r1_dir1_pre)
bus_stops_r1_dir1_pre

[1] "95"   "1891" "1915" "72"   "1892" "161"  "74"   "75"   "158"  "157" 
[11] "155"  "146"  "81"   "90"   "1486" "1899" "148"  "1731" "1797" "151" 
[21] "152"  "100"  "101"  "102"  "103"  "104"  "105"  "107"  "108"  "109" 
[31] "110"  "111"  "112"  "113"  "114"  "115"  "116"  "1912" "118"  "1914"
[41] "1358" "1582" "124"  "125"  "126"  "127"  "128"  "129"  "130"  "132" 
[51] "133"  "134"  "136"  "137"  "1487" "138"  "139"  "140"  "141"  "142" 
[61] "143"  "771"  "144"  "145"  "150"  "106"  "1913" "1488" "1684" "1560"
[71] "163"  "164"  "165"  "1361" "1589" "166"  "167"  "169"  "170"  "171" 
[81] "172"  "505"  "690"  "44"

In [140]:
for(i in bus_stops_r1_dir1_pre[1:length(bus_stops_r1_dir1_pre)]){
    print(i)
    RF_Ferns_and_Ranger('1', '1', i, 'pre')
    print('Done!')
    print('_/|_/|_')
}

[1] "95"



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




Random Ferns 

1474 samples
   7 predictor
   2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 1327, 1327, 1327, 1326, 1326, 1328, ... 
Resampling results across tuning parameters:

  depth  Accuracy   Kappa    
   1     0.9997748  0.9994711
   2     1.0000000  1.0000000
   3     1.0000000  1.0000000
   5     1.0000000  1.0000000
   6     1.0000000  1.0000000
   7     1.0000000  1.0000000
   9     1.0000000  1.0000000
  10     1.0000000  1.0000000
  11     1.0000000  1.0000000
  13     1.0000000  1.0000000
  14     1.0000000  1.0000000
  15     1.0000000  1.0000000

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was depth = 2.
Random Forest 

1474 samples
   7 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 1327, 1327, 1327, 1327, 1327, 1327, ... 
Resampling results across tuning parameters:

 


── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




Random Ferns 

776 samples
  7 predictor
  2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 699, 698, 698, 698, 698, 699, ... 
Resampling results across tuning parameters:

  depth  Accuracy   Kappa    
   1     0.9991342  0.9980658
   2     1.0000000  1.0000000
   3     1.0000000  1.0000000
   5     1.0000000  1.0000000
   6     1.0000000  1.0000000
   7     1.0000000  1.0000000
   9     1.0000000  1.0000000
  10     1.0000000  1.0000000
  11     1.0000000  1.0000000
  13     1.0000000  1.0000000
  14     1.0000000  1.0000000
  15     1.0000000  1.0000000

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was depth = 2.
Random Forest 

776 samples
  7 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 699, 698, 698, 699, 698, 699, ... 
Resampling results across tuning parameters:

  min.node.size  m

...

Random Ferns 

156 samples
  7 predictor
  2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 140, 141, 140, 141, 141, 141, ... 
Resampling results across tuning parameters:

  depth  Accuracy    Kappa       
   1     0.49875000  0.0607881222
   2     0.43861111  0.0496888104
   3     0.33680556  0.0280306832
   5     0.19250000  0.0131664253
   6     0.17305556  0.0113364824
   7     0.12972222  0.0079906797
   9     0.06583333  0.0012189013
  10     0.05736111  0.0002949853
  11     0.04625000  0.0002949853
  13     0.03750000  0.0000000000
  14     0.03750000  0.0000000000
  15     0.03750000  0.0000000000

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was depth = 1.


Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


Random Forest 

156 samples
  7 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 141, 140, 141, 140, 140, 141, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  splitrule   RMSE       Rsquared   MAE       
   1             14    extratrees  0.2066034  0.3747186  0.07248862
   1             15    maxstat     0.1814538  0.4367887  0.07204766
   2              5    variance    0.2063766  0.2099153  0.07987185
   5              6    extratrees  0.1936283  0.2624883  0.07525582
   7              6    maxstat     0.1812578  0.3049962  0.07942456
   7              9    maxstat     0.1798172  0.3625683  0.07648810
   9             10    extratrees  0.1838840  0.3563516  0.07231261
  10              1    extratrees  0.1818897  0.1754102  0.08436057
  10              3    maxstat     0.1822213  0.2678469  0.08329814
  11              5    maxstat     0.1804795  0.3122633  0.08071785
  14              2    extra


── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_logical(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_logical(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




ERROR: Error in na.fail.default(structure(list(y_clf_train = structure(c(1L, : missing values in object
